Notebook elaborado para apresentar de sismicidade global.
Criado para a disciplina Geofísica I da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Os dados utilizados nesta demonstração foram obtidos através do catálogo de eventos do [Serviço Geológico dos Estados Unidos (USGS)](https://earthquake.usgs.gov/earthquakes/search/)

Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
emerson.almeida@ufu.br<br>



<p>&nbsp;</p>
Última modificação: 11/05/2021

# Instruções iniciais

Neste notebook você irá observar uma série de eventos sísmicos ocorridos ao redor do mundo nos últimos 50 anos. Apenas os eventos com magnitude acima de 5.0 serão mostrados, para facilitar a visualização das imagens. Leia atentamente as instruções apresentadas antes de cada célula e execute-as uma a uma, para acompanhar o processo corretamente.

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação

Primeiramente vamos preparar o notebook com os módulos e funções que precisaremos no decorrer da demonstração. A célula abaixo contém as funções utilizadas nos cálculos e as funções utilizadas para gerar as figuras. Execute-a uma vez antes de executar qualquer outra parte do código e **não altere nada no código apresentado na célula** para não criar problemas na execução do código.

Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.

<p>&nbsp;</p>

**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada. 

In [ ]:
# imports e funções

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as feature
from copy import deepcopy as dc


class catalogo():
    
    def __init__(self, fname):
        self.arquivo = fname
        self.todos = pd.read_csv(fname, sep='\t', decimal=".")

        self.magnitudes = {'5.0 a 5.9': self.todos.loc[(self.todos['mag'] >=5.0) & (self.todos['mag'] < 6.0)],
                           '6.0 a 6.9': self.todos.loc[(self.todos['mag'] >=6.0) & (self.todos['mag'] < 7.0)],
                           '7.0 a 7.9': self.todos.loc[(self.todos['mag'] >=7.0) & (self.todos['mag'] < 8.0)],
                           '8.0 a 8.9': self.todos.loc[(self.todos['mag'] >=8.0) & (self.todos['mag'] < 9.0)],
                           '9.0 +': self.todos.loc[(self.todos['mag'] >=9.0)]}
        
        self.hipocentros = {'0.0 a 40.0': self.todos.loc[(self.todos['depth'] <=40)],
                            '40.0 a 100.0': self.todos.loc[(self.todos['depth'] > 40.0) & (self.todos['depth'] <= 100.0)],
                            '100.0 a 400.0': self.todos.loc[(self.todos['depth'] > 100.0) & (self.todos['depth'] <= 400.0)],
                            '400.0 a 700.0':self.todos.loc[(self.todos['depth'] > 400.0) & (self.todos['depth'] <= 700.0)]}

        
    def plota_epicentros(self, coords=None, area=None, dpi=96, res=[1280, 1024]):
        
        """
        coords = [lon_min, lon_max, lat_min, lat_max]
        
        area = [lon_min_box, lon_max_box, lat_min_box, lat_max_box]
        """
        
        
        paleta = ['black', 'blue', 'lightgreen', 'red', 'yellow']
        contornos = ['none', 'none', 'black', 'black', 'black']
        legenda = [r'$5.0 \leq M < 6.0$', r'$6.0 \leq M < 7.0$', r'$7.0 \leq M < 8.0$', \
                   r'$8.0 \leq M < 9.0$', r'$M \geq 9.0$']
        
        fig = plt.figure(figsize=(res[0]/dpi, res[1]/dpi), dpi=dpi/1.25)
        #fig = plt.figure(figsize=(14, 8))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.stock_img()
        
        for j, k in enumerate(self.magnitudes.keys()):
            curr_m = self.magnitudes[k]
            ax.scatter(curr_m['longitude'], curr_m['latitude'], edgecolors=contornos[j], s=2**(j+4), \
                       c=paleta[j], label=legenda[j])
     
        leg = ax.legend(loc='lower left', ncol=2, fontsize='large', shadow=True, borderpad=0.75, \
                        title=r'MAGNITUDE ($M$)', title_fontsize=12)
        leg._legend_box.align = "left"

        if coords:
            
            lon_min, lon_max, lat_min, lat_max = coords
            ax.set_xlim(lon_min, lon_max)
            ax.set_ylim(lat_min, lat_max)
            
        if area:
            
            lon_min_box, lon_max_box, lat_min_box, lat_max_box = area
            
            ax.plot([lon_min_box, lon_max_box], \
                    [lat_min_box + (lat_max_box - lat_min_box)/2, lat_min_box + (lat_max_box - lat_min_box)/2], \
                    '--k', linewidth=1.5)

            ax.plot([lon_min_box, lon_min_box, lon_max_box, lon_max_box, lon_min_box],
                    [lat_min_box, lat_max_box, lat_max_box, lat_min_box, lat_min_box], 'k', linewidth=2)


            ax.text(lon_min_box + lon_min_box*0.015, lat_min_box + (lat_max_box - lat_min_box)/2, 'A', \
                    horizontalalignment='center', verticalalignment='center', fontsize=16)

            ax.text(lon_max_box - lon_max_box*0.015, lat_min_box + (lat_max_box - lat_min_box)/2, 'B', \
                    horizontalalignment='center', verticalalignment='center', fontsize=16)

            
        ax.coastlines(resolution='50m', alpha=0.20)
        ax.add_feature(feature.BORDERS, linestyle='-', alpha=0.20)
        ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)

        ax.set_title('Epicentros (JAN/1971 - MAR/2021)', fontsize=14)

        # plt.tight_layout()
        plt.show()
        
        
        
    def plota_hipocentros(self, coords=None, area=None, dpi=96, res=[1280, 1024]):
        
        paleta = ['blue', 'lightgreen', 'red', 'yellow']
        contornos = ['none', 'black', 'black', 'black']
        legenda = [r'$z < 40.0$ km', r'$40.0 \leq z < 100.0$ km', r'$100.0 \leq z < 400.0$ km', \
                   r'$400.0 \leq z < 700.0$ km']
        
        fig = plt.figure(figsize=(res[0]/dpi, res[1]/dpi), dpi=dpi/1.25)
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.stock_img()
        
        for j, k in enumerate(self.hipocentros.keys()):
            curr_h = self.hipocentros[k]
            ax.scatter(curr_h['longitude'], curr_h['latitude'], edgecolors=contornos[j], s=2**(j+4), \
                       c=paleta[j], label=legenda[j])
        
        leg = ax.legend(loc='lower left', ncol=2, fontsize='large', shadow=True, borderpad=0.75, \
                        title=r'PROFUNDIDADE ($z$)', title_fontsize=12)
        leg._legend_box.align = "left"
        
        if coords:
            lon_min, lon_max, lat_min, lat_max = coords
            ax.set_xlim(lon_min, lon_max)
            ax.set_ylim(lat_min, lat_max)
            
        if area:
            lon_min_box, lon_max_box, lat_min_box, lat_max_box = area
            
            ax.plot([lon_min_box, lon_max_box], \
                    [lat_min_box + (lat_max_box - lat_min_box)/2, lat_min_box + (lat_max_box - lat_min_box)/2], \
                    '--k', linewidth=1.5)

            ax.plot([lon_min_box, lon_min_box, lon_max_box, lon_max_box, lon_min_box],
                    [lat_min_box, lat_max_box, lat_max_box, lat_min_box, lat_min_box], 'k', linewidth=2)


            ax.text(lon_min_box + lon_min_box*0.015, lat_min_box + (lat_max_box - lat_min_box)/2, 'A', \
                    horizontalalignment='center', verticalalignment='center', fontsize=16)

            ax.text(lon_max_box - lon_max_box*0.015, lat_min_box + (lat_max_box - lat_min_box)/2, 'B', \
                    horizontalalignment='center', verticalalignment='center', fontsize=16)
        
        ax.coastlines(resolution='50m', alpha=0.20)
        ax.add_feature(feature.BORDERS, linestyle='-', alpha=0.20)
        ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)

        ax.set_title('Hipocentros (JAN/1971 - MAR/2021)', fontsize=14)

        # plt.tight_layout()
        plt.show()
        
        
        
    def plota_hipocentros_perfil(self, area, dpi=96, res=[1280, 1024]):
        lon_min_box, lon_max_box, lat_min_box, lat_max_box = area
        crop = crop_area(self, [lon_min_box, lon_max_box, lat_min_box, lat_max_box])

        paleta = ['black', 'blue', 'lightgreen', 'red', 'yellow']
        contornos = ['none', 'none', 'black', 'black', 'black']
        legenda = [r'$5.0 \leq M < 6.0$', r'$6.0 \leq M < 7.0$', r'$7.0 \leq M < 8.0$', \
                   r'$8.0 \leq M < 9.0$', r'$M \geq 9.0$']
        
        fig, ax = plt.subplots(figsize=(res[0]/dpi/1.5, res[1]/dpi/2), dpi=dpi/1.25)
        
        for j, k in enumerate(crop.magnitudes.keys()):
            curr_h = crop.magnitudes[k]
            ax.scatter(curr_h['longitude'], curr_h['depth'], edgecolors=contornos[j], s=2**(j+4), \
                       c=paleta[j], label=legenda[j])
        
        leg = ax.legend(loc='lower left', ncol=2, fontsize='large', shadow=True, borderpad=0.75, \
                        title=r'MAGNITUDE ($M$)', title_fontsize=12)
        leg._legend_box.align = "left"
                
        ax.set_xlabel('Longitude (°)', fontsize=14)
        ax.set_ylabel('Profundidade (km)', fontsize=14)
        ax.set_xlim([lon_min_box, lon_max_box])
        ax.set_ylim([0, 800])
        ax.invert_yaxis()
        ax.set_title('Hipocentros próximos ao perfil A-B', fontsize=14)
        
        plt.show()
        
        
def crop_area(cat_in, area):
    cat_out = dc(cat_in)
    lon_min_box, lon_max_box, lat_min_box, lat_max_box = area
    
    cat_out.todos = cat_in.todos[(cat_in.todos['longitude'] >= lon_min_box) &
                                 (cat_in.todos['longitude'] <= lon_max_box) &
                                 (cat_in.todos['latitude'] >= lat_min_box) &
                                 (cat_in.todos['latitude'] <= lat_max_box)]
    
    cat_out.magnitudes = {'5.0 a 5.9': cat_out.todos.loc[(cat_out.todos['mag'] >=5.0) & (cat_out.todos['mag'] < 6.0)],
                          '6.0 a 6.9': cat_out.todos.loc[(cat_out.todos['mag'] >=6.0) & (cat_out.todos['mag'] < 7.0)],
                          '7.0 a 7.9': cat_out.todos.loc[(cat_out.todos['mag'] >=7.0) & (cat_out.todos['mag'] < 8.0)],
                          '8.0 a 8.9': cat_out.todos.loc[(cat_out.todos['mag'] >=8.0) & (cat_out.todos['mag'] < 9.0)],
                          '9.0 +': cat_out.todos.loc[(cat_out.todos['mag'] >=9.0)]}
        
    cat_out.hipocentros = {'0.0 a 40.0': cat_out.todos.loc[(cat_out.todos['depth'] <=40)],
                           '40.0 a 100.0': cat_out.todos.loc[(cat_out.todos['depth'] > 40.0) & (cat_out.todos['depth'] <= 100.0)],
                           '100.0 a 400.0': cat_out.todos.loc[(cat_out.todos['depth'] > 100.0) & (cat_out.todos['depth'] <= 400.0)],
                           '400.0 a 700.0':cat_out.todos.loc[(cat_out.todos['depth'] > 400.0) & (cat_out.todos['depth'] <= 700.0)]}

    return cat_out

### Passo 02 - Carregar os dados

Esta demonstração utilizará dados carregados a partir de um arquivo ASCII em formato .csv contendo uma tabela com os eventos de magnitude acima de 5.0 registrados no catálogo da USGS entre janeiro de 1971 e março de 2021. Execute a célula abaixo para carregar este arquivo.

In [ ]:
usgs = catalogo('usgs.csv')

### Passo 03 - Mapa de Epicentros

Os eventos sísmicos ocorrem ao redor de todo o mundo, porém são mais frequentes em algumas regiões específicas. Isto pode ser observado pelo mapa de epicentros, que são os pontos na superfície da Terra que encontra-se exatamente acima do local de ocorrência do terremoto.

Execute a célula abaixo para gerar um mapa de epicentros dos eventos com magnitude $M \geq 5.0$ registrados entre os anos de 1971 e 2021.

In [ ]:
usgs.plota_epicentros()

Você consegue perceber o padrão na localização dos epicentros de maiores magnitudes? Onde concentram-se os eventos de menores e maiores magnitudes?

### Passo 04 - Mapa de Hipocentros

O hipocentro, ou foco, representa o local em profundidade onde a liberação de energia do evento sísmico de fato ocorreu. Os epicentros vistos no mapa anterior são uma projeção em superfície dos hipocentros. Assim como os epicentros concentram-se em locais específicos do globo, os hipocentros concentram-se em profundidades específicas dependendo da região em que os eventos ocorrem.

Execute a célula abaixo para gerar um mapa de hipocentros dos eventos apresentados no mapa anterior.

In [ ]:
usgs.plota_hipocentros()

Você percebe algum padrão em relação à profundidade em que os eventos ocorreram, comparando esta informação com o mapa de Epicentros?

### Passo 05 - Mapas da América do Sul

Vamos agora analisar especificamente a América do Sul, limitando a exibição dos mapas à região compreendida entre as longitudes -90.0º e -30.0º e entre as latitudes -70.0º e 20.0º.

Execute a célula abaixo para limitar esta região.

In [ ]:
regiao = [-90.0, -30.0, -70.0, 20.0]

Adicionalmente vamos especificar também uma subárea na América do Sul, correspondente a uma pequena porção dos Andes, que será analisada posteriormente.

In [ ]:
selecao = [-75.0, -61.0, -30.0, -20.0]

Agora você irá gerar o mapa propriamente dito, incluindo a delimitação da subárea configurada na célula acima. Execute a célula abaixo para gerá-lo.

In [ ]:
usgs.plota_epicentros(coords=regiao, area=selecao)

Como você descreveria a concentração dos epicentros no continente sul-americano?

Execute a célula abaixo para gerar um mapa de hipocentros dos eventos registrados no continente e compare-o com o mapa de epicentros.

In [ ]:
usgs.plota_hipocentros(coords=regiao, area=selecao)

Você deve ter observado que a subárea delimitada na região andina apresenta um perfil central A-B orientado na direção leste-oeste. Execute a célula abaixo para gerar uma visualização acompanhando este perfil e que irá apresentar todos os hipocentros presentes no interior desta subárea.

In [ ]:
usgs.plota_hipocentros_perfil(area=selecao)

O que isso te diz sobre a dinâmica geológica desta região?

Volte à segunda célula do Passo 05, modifique os limites da subárea através da qual passa o perfil A-B e verifique se este comportamento se repete. Você pode inserir as coordenadas-limite que você quiser ou pode utilizar as sugestões abaixo copiando as coordenadas desejadas e colando-as na célula indicada.

**Sugestão 1:** selecao = [-80.0, -60.0, -40.0, -30.0]

**Sugestão 2:** selecao = [-85.0, -70.0, -10.0, 0.0]